In [ ]:
pip install pydicom nibabel numpy torch torchvision segmentation-models-pytorch scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install albumentations

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import numpy as np
import pydicom
import nibabel as nib
import torch
from torch.utils.data import DataLoader, Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import pandas as pd
import random
import segmentation_models_pytorch as smp
import torch.optim as optim
import torch.nn as nn
from scipy.ndimage import zoom
from operator import itemgetter
from sklearn.metrics import jaccard_score

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

num_classes = 9
batch_size = 8
img_root = "/home/ealam/JHIR_Hip_Knee_Datasets/Knee/Images"
mask_root = "/home/ealam/JHIR_Hip_Knee_Datasets/Knee/Annotations"
metadata_path = "/home/ealam/JHIR_Hip_Knee_Datasets/Knee/segmentation_with_racegender.csv"
test_augmentations = A.Compose([
    A.Resize(height=256, width=256),
    A.Normalize(mean=(0.485,), std=(0.229,)),
    ToTensorV2(),
])

num_epochs = 100

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class MulticlassHipSegmentationDataset(Dataset):
    def __init__(self, img_root, mask_root, metadata_df, paired_files, num_classes, transforms=None, preprocessing=None):
        self.img_root = img_root
        self.mask_root = mask_root
        self.metadata_df = metadata_df
        self.paired_files = paired_files
        self.num_classes = num_classes
        self.transforms = transforms
        self.preprocessing = preprocessing

    def __len__(self):
        return len(self.paired_files)

    def __getitem__(self, idx):
        image_file, mask_file = self.paired_files[idx]
        if not os.path.exists(os.path.join(self.mask_root, mask_file)):
            return None

        dicom_image = pydicom.dcmread(os.path.join(self.img_root, image_file))
        image = dicom_image.pixel_array.astype(np.float32)
        annotation = nib.load(os.path.join(self.mask_root, mask_file))
        annotation_data = annotation.get_fdata()
        if len(annotation_data.shape) == 3:
            annotation_data = annotation_data[:, :, 0]

        annotation_data = self.calculate_flipped_rotated_mask(annotation_data)

        if annotation_data.ndim > 2 and annotation_data.shape[-1] != 1:
            raise ValueError('Mask has multiple channels')

        if image.shape != annotation_data.shape:
            zoom_factors = np.array(image.shape) / np.array(annotation_data.shape)
            annotation_data = zoom(annotation_data, zoom_factors, order=0)

        if self.transforms is not None:
            transformed = self.transforms(image=image, mask=annotation_data)
            image = transformed["image"]
            annotation_data = transformed["mask"]

        annotation_data_onehot = self.one_hot_encode(annotation_data)

        if self.preprocessing is not None:
            transformed = self.preprocessing(image=image, mask=annotation_data_onehot)
            image = transformed["image"]
            annotation_data_onehot = transformed["mask"]

        patient_id = int(float(image_file.split(".")[0]))
        racegender_info = self.metadata_df.loc[self.metadata_df['id'] == patient_id]['racegender'].values
        racegender = racegender_info[0] if racegender_info else 'Unknown'

        return image, annotation_data_onehot, racegender

    def one_hot_encode(self, mask):
        one_hot_mask = np.zeros((self.num_classes, *mask.shape), dtype=np.float32)
        for class_idx in range(self.num_classes):
            one_hot_mask[class_idx][mask == class_idx] = 1.0
        return one_hot_mask

    def calculate_flipped_rotated_mask(self, mask):
        rotated_mask = cv2.rotate(mask, cv2.ROTATE_90_CLOCKWISE)
        flipped_rotated_mask = cv2.flip(rotated_mask, 1)
        return flipped_rotated_mask

metadata_df = pd.read_csv(metadata_path)

image_files = sorted(os.listdir(img_root))
mask_files = sorted(os.listdir(mask_root))

paired_files = []

for image_file in image_files:
    image_id = os.path.splitext(image_file)[0]
    mask_file = f"{image_id}.nii.gz"
    if mask_file in mask_files:
        paired_files.append((image_file, mask_file))

random.shuffle(paired_files)

train_size = int(0.7 * len(paired_files))
valid_size = int(0.1 * len(paired_files))
test_size = len(paired_files) - train_size - valid_size

train_pairs = paired_files[:train_size]
valid_pairs = paired_files[train_size:train_size + valid_size]
test_pairs = paired_files[train_size + valid_size:]

# Create datasets and dataloaders for each racegender combination

# Filter racegender MB and MW
train_set_MB_MW = MulticlassHipSegmentationDataset(
    img_root, mask_root, metadata_df[metadata_df['racegender'].isin(['MB', 'MW'])], train_pairs,
    num_classes, transforms=test_augmentations
)
valid_set_MB_MW = MulticlassHipSegmentationDataset(
    img_root, mask_root, metadata_df[metadata_df['racegender'].isin(['MB', 'MW'])], valid_pairs,
    num_classes, transforms=test_augmentations
)

# Filter racegender FB and FW
train_set_FB_FW = MulticlassHipSegmentationDataset(
    img_root, mask_root, metadata_df[metadata_df['racegender'].isin(['FB', 'FW'])], train_pairs,
    num_classes, transforms=test_augmentations
)
valid_set_FB_FW = MulticlassHipSegmentationDataset(
    img_root, mask_root, metadata_df[metadata_df['racegender'].isin(['FB', 'FW'])], valid_pairs,
    num_classes, transforms=test_augmentations
)

# Filter racegender MW and FW
train_set_MW_FW = MulticlassHipSegmentationDataset(
    img_root, mask_root, metadata_df[metadata_df['racegender'].isin(['MW', 'FW'])], train_pairs,
    num_classes, transforms=test_augmentations
)
valid_set_MW_FW = MulticlassHipSegmentationDataset(
    img_root, mask_root, metadata_df[metadata_df['racegender'].isin(['MW', 'FW'])], valid_pairs,
    num_classes, transforms=test_augmentations
)

# Filter racegender MB and FB
train_set_MB_FB = MulticlassHipSegmentationDataset(
    img_root, mask_root, metadata_df[metadata_df['racegender'].isin(['MB', 'FB'])], train_pairs,
    num_classes, transforms=test_augmentations
)
valid_set_MB_FB = MulticlassHipSegmentationDataset(
    img_root, mask_root, metadata_df[metadata_df['racegender'].isin(['MB', 'FB'])], valid_pairs,
    num_classes, transforms=test_augmentations
)

# Define model architecture, optimizer, and criterion for each model
models = []
optimizers = []
criterions = []
for _ in range(4):
    model = smp.Unet(
        encoder_name="resnet18",
        encoder_weights="imagenet",
        in_channels=1,
        classes=num_classes,
    )
    model.to(device)
    models.append(model)
    optimizers.append(optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5))
    criterions.append(nn.CrossEntropyLoss())

# Create datasets and dataloaders for each racegender combination
train_sets = [train_set_MB_MW, train_set_FB_FW, train_set_MW_FW, train_set_MB_FB]
valid_sets = [valid_set_MB_MW, valid_set_FB_FW, valid_set_MW_FW, valid_set_MB_FB]

# Training loop for each model
for idx, (train_set, valid_set) in enumerate(zip(train_sets, valid_sets)):
    print(f"Training Model {idx + 1}...")
    model = models[idx]
    optimizer = optimizers[idx]
    criterion = criterions[idx]

    for epoch in range(num_epochs):
        train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
        valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=False, num_workers=2)

        model.train()
        iou_train_list = []

        for batch_idx, (images, masks, _) in enumerate(train_loader):
            if images is None:
                continue

            images, masks = images.to(device), masks.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            predicted_masks = torch.argmax(outputs, dim=1)

            train_iou = jaccard_score(
                masks.argmax(dim=1).cpu().numpy().flatten(),
                predicted_masks.cpu().numpy().flatten(),
                average='micro'
            )
            iou_train_list.append(train_iou)

            loss = criterion(outputs, masks.argmax(dim=1))
            loss.backward()
            optimizer.step()

        train_iou_avg = np.mean(iou_train_list)

        model.eval()
        iou_valid_list = []

        for batch_idx, (images, masks, _) in enumerate(valid_loader):
            images, masks = images.to(device), masks.to(device)
            with torch.no_grad():
                outputs = model(images)
            predicted_masks = torch.argmax(outputs, dim=1)

            valid_iou = jaccard_score(
                masks.argmax(dim=1).cpu().numpy().flatten(),
                predicted_masks.cpu().numpy().flatten(),
                average='micro'
            )
            iou_valid_list.append(valid_iou)

        valid_iou_avg = np.mean(iou_valid_list)

        print(f"Epoch [{epoch + 1}/{num_epochs}] - Train IoU: {train_iou_avg:.4f} - Validation IoU: {valid_iou_avg:.4f}")

    print(f"Training Model {idx + 1} complete.")


Training Model 1...


/tmp/ipykernel_74967/1761624781.py:87: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'
/tmp/ipykernel_74967/1761624781.py:87: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'
/home/ealam/MDCL/lib/python3.10/site-packages/pydicom/charset.py:743: UserWarning: Incorrect value for Specific Character Set 'ISO_2022_IR_6' - assuming 'ISO 2022 IR 6'
  _warn_about_invalid_encoding(encoding, patched)
/tmp/ipykernel_74967/1761624781.py:87: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to c

Epoch [1/100] - Train IoU: 0.0424 - Validation IoU: 0.0765


/tmp/ipykernel_74967/1761624781.py:87: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'
/tmp/ipykernel_74967/1761624781.py:87: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'
/home/ealam/MDCL/lib/python3.10/site-packages/pydicom/charset.py:743: UserWarning: Incorrect value for Specific Character Set 'ISO_2022_IR_6' - assuming 'ISO 2022 IR 6'
  _warn_about_invalid_encoding(encoding, patched)
/tmp/ipykernel_74967/1761624781.py:87: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to c

Epoch [2/100] - Train IoU: 0.0565 - Validation IoU: 0.0405


...

Epoch [98/100] - Train IoU: 0.9515 - Validation IoU: 0.8647


/tmp/ipykernel_74967/1761624781.py:87: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'
/tmp/ipykernel_74967/1761624781.py:87: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'
/home/ealam/MDCL/lib/python3.10/site-packages/pydicom/charset.py:743: UserWarning: Incorrect value for Specific Character Set 'ISO_2022_IR_6' - assuming 'ISO 2022 IR 6'
  _warn_about_invalid_encoding(encoding, patched)
/tmp/ipykernel_74967/1761624781.py:87: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to c

Epoch [99/100] - Train IoU: 0.9475 - Validation IoU: 0.8650


/tmp/ipykernel_74967/1761624781.py:87: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'
/tmp/ipykernel_74967/1761624781.py:87: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  racegender = racegender_info[0] if racegender_info else 'Unknown'
/home/ealam/MDCL/lib/python3.10/site-packages/pydicom/charset.py:743: UserWarning: Incorrect value for Specific Character Set 'ISO_2022_IR_6' - assuming 'ISO 2022 IR 6'
  _warn_about_invalid_encoding(encoding, patched)
/tmp/ipykernel_74967/1761624781.py:87: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to c

Epoch [100/100] - Train IoU: 0.9516 - Validation IoU: 0.8629
Training Model 4 complete.


In [ ]:
# Specify the directory where models will be saved
save_dir = "/home/ealam/MDCL"

# Save models with specified format
for idx, (model, test_set) in enumerate(zip(models, test_sets)):
    racegenders_involved = ["MB_MW", "FB_FW", "MW_FW", "MB_FB"][idx]
    model_type = "Knee"  # Change this to "Knee" if applicable
    model_name = f"{racegenders_involved}_{model_type}_baseline.pth"
    save_path = os.path.join(save_dir, model_name)
    torch.save({
        'racegenders_involved': racegenders_involved,
        'model_type': model_type,
        'state_dict': model.state_dict()
    }, save_path)

    print(f"Model saved: {save_path}")


Model saved: /home/ealam/MDCL/MB_MW_Knee_baseline.pth
Model saved: /home/ealam/MDCL/FB_FW_Knee_baseline.pth
Model saved: /home/ealam/MDCL/MW_FW_Knee_baseline.pth
Model saved: /home/ealam/MDCL/MB_FB_Knee_baseline.pth
